In [3]:
from rdkit import Chem, Geometry
from crest_confs_utils import (get_low_energy_conformer,
                               xtbError,
                               display_3d_mol,
                               xtb_single_point)

In [4]:
import subprocess
import tempfile

In [19]:
def crest_conf_search(input_mol: Chem.rdchem.Mol,
                 charge: int = 0,
                 e_state: int = 0,
                 solvent: str = '',
                 mmff_max_iters: int = 200) -> Chem.rdchem.Mol:
    # makes sure that there is a conformer embedded in mol
    mol = Chem.rdchem.Mol(input_mol)
    if len(mol.GetConformers()) == 0:
        mol = get_low_energy_conformer(mol, mmff_max_iters)

    xtb_xyz = ''
    # runs calculations in tmp directory
    with tempfile.TemporaryDirectory() as tmp:
        # create .xyz file in the tmp directory
        Chem.rdmolfiles.MolToXYZFile(mol, f'{tmp}/input.xyz')
        # run xtb on the input file
        xtb_args = ['-c', str(charge), '-u', str(e_state)]
        if solvent != '':
            xtb_args += ['-g', solvent]
        # proc = subprocess.run(['crest', 'input.xyz'] + xtb_args, 
        proc = subprocess.run(['crest', 'input.xyz'], 
                              cwd=tmp,
                            #   stdout=subprocess.PIPE,
                              stdout=None, 
                              stderr=subprocess.DEVNULL)
        # if proc.returncode != 0:
        #     raise xtbError('xtb abnormal termination')
        # with open(f'{tmp}/xtbopt.xyz') as file:
        #     # first two lines of xyz are atom count and comments
        #     # last line is blank
        #     xtb_xyz = file.read().split('\n')[2:len(xtb_xyz)-1]

    # # creates a new RDKit Mol with embedded conformer from the xtb xyz output
    # mol.RemoveAllConformers()
    # conf = Chem.rdchem.Conformer(mol.GetNumAtoms())
    # for i, line in enumerate(xtb_xyz):
    #     ls = line.split()
    #     x, y, z = float(ls[1]), float(ls[2]), float(ls[3])
    #     conf.SetAtomPosition(i, Geometry.rdGeometry.Point3D(x, y, z))
    # mol.AddConformer(conf)
    # return mol

In [27]:
m = Chem.MolFromSmiles('O')
m = get_low_energy_conformer(m)
display_3d_mol(m)
print(Chem.MolToXYZBlock(m))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

3

O      0.005785    0.397773    0.000000
H     -0.766280   -0.187784    0.000000
H      0.760495   -0.209989    0.000000



In [18]:
# m = Chem.MolFromSmiles('OC(CN1CCN(C2=C(F)C=NC=C2F)CC1)=O')
m = Chem.MolFromSmiles('O')
m = crest_conf_search(m)


10

O      1.254173    0.729445   -0.388358
C      0.547613   -0.343848    0.236398
C     -0.868274   -0.383902   -0.324814
O     -1.512685    0.863387   -0.071621
H      2.146978    0.740096   -0.000482
H      0.529984   -0.148747    1.313038
H      1.074175   -1.281358    0.037274
H     -0.854420   -0.533840   -1.408834
H     -1.454454   -1.181005    0.140235
H     -0.863089    1.539772   -0.349299


       |                                            |
       |                 C R E S T                  |
       |                                            |
       |  Conformer-Rotamer Ensemble Sampling Tool  |
       |          based on the GFN methods          |
       |             P.Pracht, S.Grimme             |
       |          Universitaet Bonn, MCTC           |
       Version 2.11.2, Fr 17. Dec 12:10:44 CEST 2021
  Using the xTB program. Compatible with xTB version 6.4.0

   Cite work conducted with this code as

   P. Pracht, F. Bohle, S. Grimme, PCCP, 2020, 22, 7169-7192.